In [7]:
import cv2
from cvzone.HandTrackingModule import HandDetector

In [3]:
detector = HandDetector(detectionCon=0.7, maxHands=2)

In [10]:
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

while True:
    success, img = cap.read()
    if not success:
        break

    hands, img = detector.findHands(img)

    if len(hands) == 2:
        hand1, hand2 = hands
        if hand1["center"][0] < hand2["center"][0]:
            left_hand = hand1
            right_hand = hand2
        else:
            left_hand = hand2
            right_hand = hand1

        fingers_left = detector.fingersUp(left_hand)
        freq_band = "none"
        if fingers_left == [0, 0, 0, 0, 0]:
            freq_band = "bass"
        elif fingers_left == [0, 1, 0, 0, 0]:
            freq_band = "mid"
        elif fingers_left == [0, 0, 0, 0, 1, ]:
            freq_band = "treble"

        if freq_band != "none":
            distance, info, img = detector.findDistance(
                right_hand["lmList"][8][:-1], right_hand["lmList"][4][:-1], img)
            gain = max(0, min(100, int(distance * 0.5)))
        else:
            gain = 0

        cv2.putText(img, f"Freq Band: {freq_band}", (10, 70),
                    cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3,)
        cv2.putText(img, f"Gain (Distance): {gain}", (10, 110),
                    cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3,)

    cv2.imshow("Hand Gesture Equalizer Control", img)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()